### Import Required Packages

In [1]:
%load_ext cython

In [2]:
import os
import sys
import git

import numpy as np
import pandas as pd
pd.set_option('display.float_format', '{:.3f}'.format)

#### Put the Main Package Library on the PYTHONPATH

In [3]:
git_repo = git.Repo('.', search_parent_directories=True)
git_root = git_repo.git.rev_parse('--show-toplevel')

sys.path[0] = git_root
sys.path[0]

'/Users/ericlundquist/Repos/rankfm'

#### Dynamically Re-Load all Package Modules

In [4]:
%load_ext autoreload
%autoreload 2

from rankfm.rankfm import RankFM
from rankfm.evaluation import hit_rate, reciprocal_rank, discounted_cumulative_gain, precision, recall, diversity

#### Set File Path Constants

In [5]:
repo_root = "/Users/ericlundquist/Repos/rankfm"
data_path = os.path.join(repo_root, "data/examples")
print("\n".join([repo_root, data_path]))

/Users/ericlundquist/Repos/rankfm
/Users/ericlundquist/Repos/rankfm/data/examples


### Load Example Data

#### Load Interactions Data

In [6]:
raw_interactions = pd.read_csv(os.path.join(data_path, 'ML_1M_RATINGS.csv'))[['user_id', 'item_id']]
raw_interactions = raw_interactions.astype(np.int32)
raw_interactions['user_id'] = raw_interactions['user_id'] - 1
raw_interactions['item_id'] = raw_interactions['item_id'] - 1
raw_interactions.head()

,user_id,item_id
0,0,1192
1,0,660
2,0,913
3,0,3407
4,0,2354


In [7]:
raw_interactions = raw_interactions.astype(np.int32).values
print(raw_interactions.dtype)
print(raw_interactions[:5])

int32
[[   0 1192]
 [   0  660]
 [   0  913]
 [   0 3407]
 [   0 2354]]


### Initialize Internal Data

In [8]:
model = RankFM(factors=20, loss='warp', max_samples=10, regularization=0.01, sigma=0.1, learning_rate=0.10, learning_schedule='constant')
model

In [9]:
model._init_all(raw_interactions)
model

/Users/ericlundquist/anaconda3/lib/python3.7/site-packages/numba/typed/typeddict.py:34: NumbaTypeSafetyWarning: unsafe cast from int64 to int32. Precision may be lost.
  d[key] = value


In [10]:
model.user_items_py[0]

array([   0,   47,  144,  253,  513,  517,  574,  580,  581,  593,  639,
        689,  708,  740,  853,  858,  877,  957,  963,  964,  970, 1025,
       1104, 1107, 1117, 1154, 1178, 1195, 1421, 1439, 1574, 1658, 1727,
       1781, 1782, 1838, 1848, 2102, 2128, 2147, 2162, 2205, 2483, 2488,
       2557, 2586, 2592, 2599, 2710, 2889, 2898, 2969, 3177], dtype=int32)

### Define a Cython Fit Function

In [18]:
%%cython --annotate
# cython: language_level=3, boundscheck=False, wraparound=False, cdivision=True, profile=True

from libc.stdlib cimport rand
from libc.math cimport log, exp, pow
from scipy.linalg.cython_blas cimport sdot

import cython
from cython cimport floating, integral

import numpy as np


# define linear algebra wrapper functions
# ---------------------------------------


cdef void add_vv(int n, float *x, float *y, float *out):
    """add two vectors and place the result in a third"""
    
    cdef int i
    for i in range(n):
        out[i] = x[i] + y[i]
        
        
cdef void sub_vv(int n, float *x, float *y, float *out):
    """add two vectors and place the result in a third"""
    
    cdef int i
    for i in range(n):
        out[i] = x[i] - y[i]
        
        
cdef float dot_vv(float[::1] x, float[::1] y):
    """compute a vector-vector dot product"""
    
    # set the vector length and stride
    cdef int n = x.shape[0], incx = 1, incy = 1
    
    # call the underlying BLAS routine and return the scalar result
    return sdot(&n, &x[0], &incx, &y[0], &incy)
        
        
# define main fit function
# ------------------------


def fit(
    int[:, :] interactions,
    float[:] sample_weight,
    float[:, :] x_uf,
    float[:, :] x_if,
    float[:] w_i,
    float[:] w_if,
    float[:, ::1] v_u,
    float[:, ::1] v_i,
    float[:, :] v_uf,
    float[:, :] v_if
):
    
    cdef int margin = 1
    cdef int N, U, I, P, Q, F
    cdef int x_uf_any, x_if_any
    cdef int epoch
    cdef float eta
    cdef float log_likelihood
    cdef int r, row, sampled
    cdef int u, i, j
    cdef float sw
    cdef int min_index
    cdef float pairwise_utility, min_pairwise_utility
    
    cdef int one = 1
    cdef int max_samples = 10
     
    # calculate matrix shapes
    N = interactions.shape[0]
    U = v_u.shape[0]
    I = v_i.shape[0]
    P = v_uf.shape[0]
    Q = v_if.shape[0]
    F = v_u.shape[1]
    
#     # define pointers to relevant model weight matrix slices
#     cdef floating * user
#     cdef floating * liked
#     cdef floating * disliked
    
    # create temporary memoryviews to hold intermediate results
    cdef float[:] f_ij = np.empty(F, dtype=np.float32)
    
    # determine whether user-features/item-features were supplied
    x_uf_any = int(np.asarray(x_uf).any())
    x_if_any = int(np.asarray(x_if).any())
    
    # create a shuffle index to diversify each training epoch
    shuffle_index = np.arange(N, dtype=np.int32)
    cdef int[:] shuffle_index_mv = shuffle_index
    
    #############################
    ### START EPOCH LOOP HERE ###
    #############################
    
    # set the learning rate for this epoch
    eta = 0.10 / pow(2, 0.25)
    
    # re-shuffle the data prior to training
    np.random.shuffle(shuffle_index)
    log_likelihood = 0.0
    
    print("N:", N)
    for r in range(N):
        
        # locate the user/item/weight
        row = shuffle_index_mv[r]
        u = int(interactions[row, 0])
        i = int(interactions[row, 1])
        sw = sample_weight[row]
        
        # record the index and the pairwise utility of the worst rank reversal
        min_index = -1
        min_pairwise_utility = 1e6
        
        # start the WARP sampling loop for the (u, i) pair
        for sampled in range(1, max_samples + 1):
            
            # randomly sample an unobserved item (j) for the user
            while True:
                j = rand() % I
                # NOTE: bring in the user-items search here
                break
                
            # take the vector difference of item factors: v_i(i) - v_i(j)
            sub_vv(F, &v_i[i][0], &v_i[j][0], &f_ij[0])
                
            # calculate the pairwise utility score for the (u, i, j) triplet
            pairwise_utility = 0.0
            pairwise_utility += w_i[i] - w_i[j] 
            pairwise_utility += sdot(&F, &v_u[u][0], &one, &f_ij[0], &one) 
            
            # check to see if this (u, i, j) triple has the worst rank reversal seen so far
            if pairwise_utility < min_pairwise_utility:
                min_index = j
                min_pairwise_utility = pairwise_utility
                
            # check to see if this (u, i, j) triple violates the pairwise utility margin
            if pairwise_utility < margin:
                break
                
        # set the final sampled negative item index and calculate the WARP multiplier
        j = min_index
        pairwise_utility = min_pairwise_utility
        multiplier = log((I - 1) / sampled) / log(I)
        log_likelihood += log(1 / (1 + exp(-pairwise_utility)))

        print("r", r, "row", row, "u", u, "i", i, "sampled", sampled, "j", j)
        print("pairwise_utility", pairwise_utility, "multiplier", multiplier, "log_likelihood", log_likelihood)
            


#### Call the Fit Function for Testing

In [19]:
interactions = model.interactions[:5]
sample_weight = model.sample_weight[:5]
x_uf = model.x_uf
x_if = model.x_if
w_i = model.w_i
w_if = model.w_if
v_u = model.v_u
v_i = model.v_i
v_uf = model.v_uf
v_if = model.v_if

In [20]:
interactions.shape, sample_weight.shape

((5, 2), (5,))

In [21]:
fit(
    interactions,
    sample_weight,
    x_uf,
    x_if,
    w_i,
    w_if,
    v_u,
    v_i,
    v_uf,
    v_if
)

N: 5
r 0 row 4 u 0 i 2162 sampled 1 j 744
pairwise_utility 0.08637633919715881 multiplier 0.9999671600521997 log_likelihood -0.6508913040161133
r 1 row 2 u 0 i 853 sampled 1 j 3519
pairwise_utility 0.04956848546862602 multiplier 0.9999671600521997 log_likelihood -1.3195613622665405
r 2 row 3 u 0 i 3177 sampled 1 j 2863
pairwise_utility -0.005568861961364746 multiplier 0.9999671600521997 log_likelihood -2.0154969692230225
r 3 row 0 u 0 i 1104 sampled 1 j 397
pairwise_utility -0.08537043631076813 multiplier 0.9999671600521997 log_likelihood -2.7522401809692383
r 4 row 1 u 0 i 639 sampled 1 j 2969
pairwise_utility 0.05688032507896423 multiplier 0.9999671600521997 log_likelihood -3.417351484298706


In [23]:
u = 0
i = 853
j = 3519

pairwise_utility = 0.0
pairwise_utility += w_i[i] - w_i[j]
pairwise_utility += np.dot(v_i[i] - v_i[j], v_u[u])
pairwise_utility

0.04956848546862602

### Define Cython Helper Functions

#### Random Number Generation

In [ ]:
%%cython --annotate
# cython: language_level=3, boundscheck=False, wraparound=False, cdivision=True

import cython
from libc.stdlib cimport rand

@cython.cdivision(True)
cdef int random_item(int I):
    """sample a random item"""
    
    return rand() % I


#### Linear Algebra

In [ ]:
%%cython --annotate
# cython: language_level=3, boundscheck=False, wraparound=False, cdivision=True

import cython
from scipy.linalg.cython_blas cimport sdot, sgemv
import numpy as np


cdef void add_vv(float *x, float *y, float *out, int n):
    """add two vectors and place the result in a third"""
    
    cdef int i
    for i in range(n):
        out[i] = x[i] + y[i]
        
        
cdef void sub_vv(float *x, float *y, float *out, int n):
    """add two vectors and place the result in a third"""
    
    cdef int i
    for i in range(n):
        out[i] = x[i] - y[i]
        
    
        

# def process(float[::1] x, float[::1] y):
#     """test math functions"""
    
#     # declare local variables
#     cdef int one = 1, n = x.shape[0]
#     cdef float[:] out = np.empty(n, dtype=np.float32)
#     cdef float res
    
#     # perform calculation
#     add_vv(&x[0], &y[0], &out[0], n)
    
#     # calculate a dot product
#     res = sdot(&n, &x[0], &one, &y[0], &one)
    
#     return res
#     # return np.asarray(out)

In [ ]:
x = np.array([1, 2, 3], dtype=np.float32)
y = np.array([3, 6, 9], dtype=np.float32)
print(x)
print(y)

In [ ]:
process(x, y)

### Look into the Built-In Random Number Generator
* rand() gives a random integer `[0, RAND_MAX]`
* RAND_MAX is typically the max 32-bit integer value or `2,147,483,647`

In [ ]:
%%cython --annotate

import cython
from libc.stdlib cimport rand, RAND_MAX
print("RAND_MAX:", RAND_MAX)

@cython.cdivision(True)
cpdef int random_item(int n_items):
    """sample a random item"""
    
    return rand() % n_items


In [ ]:
# %%time
random_items = pd.Series([random_item(20) for _ in range(int(1e6))])
random_items.value_counts().sort_index()

### Write Simple Utilities for Vector Addition/Subtraction

In [ ]:
%%cython --annotate

import cython

@cython.wraparound(False)
@cython.boundscheck(False)
cpdef void add_vv(float[::1] x, float[::1] y, float[::1] z):
    """add two vectors and put the result in a third"""
    
    cdef int i
    cdef int n = x.shape[0]
    for i in range(n):
        z[i] = x[i] + y[i]
        
@cython.wraparound(False)
@cython.boundscheck(False)
cpdef void sub_vv(float[::1] x, float[::1] y, float[::1] z):
    """subtract two vectors and put the result in a third"""
    
    cdef int i
    cdef int n = x.shape[0]
    for i in range(n):
        z[i] = x[i] - y[i]

In [ ]:
x = np.random.random(50).astype(np.float32)
y = np.random.random(50).astype(np.float32)
z = np.zeros(50).astype(np.float32)

In [ ]:
%%timeit
x + y

In [ ]:
%%timeit
add_vv(x, y, z)
# np.asarray(z)

In [ ]:
x - y

In [ ]:
# %%timeit
sub_vv(x, y, z)
np.asarray(z)

### Try to Get BLAS Wrappers Working

#### Wrap Vector-Vector and Matrix-Vector Dot Product Functions

In [ ]:
%%cython

from scipy.linalg.cython_blas cimport sdot, sgemv

cpdef float dot_vv(float[::1] x, float[::1] y):
    """compute a vector-vector dot product"""
    
    # set the vector length and stride
    cdef int n = x.shape[0], incx = 1, incy = 1
    
    # call the underlying BLAS routine and return the scalar result
    return sdot(&n, &x[0], &incx, &y[0], &incy)


cpdef void dot_mv(float[::1, :] A, float[::1] x, float[::1] y):
    """compute a matrix-vector dot product"""
        
    # set the matrix dimensions
    cdef int m = A.shape[0], n = A.shape[1]
    
    # set the alpha/beta scalars
    cdef float alpha = 1.0, beta = 0.0
    
    # set the x/y stride
    cdef int incx = 1, incy = 1
    
    # call the underlying BLAS routine and store result in [y]
    # NOTE: the matrix [A] must be F-CONTIGUOUS for this to work as written
    # NOTE: it would be much better to figure out how to get this to work C-CONTIGUOUS
    # NOTE: the result is placed into the vector [y] instead of being returned
    sgemv('N', &m, &n, &alpha, &A[0, 0], &m, &x[0], &incx, &beta, &y[0], &incy)




#### Confirm the Accuracy of the Results

##### vector-vector dot product

In [ ]:
x = np.random.randn(50).astype(np.float32)
y = np.random.randn(50).astype(np.float32)

In [ ]:
%%timeit
np.dot(x, y)

In [ ]:
%%timeit
dot_vv(x, y)

##### matrix-vector dot product

In [ ]:
A = np.random.random(10000).reshape((1000, 10)).T.astype(np.float32)
x = np.random.random(1000).astype(np.float32)
y = np.zeros(10, dtype=np.float32)

print(A.shape)
print(x.shape)
print(y.shape)
A.flags

In [ ]:
%%timeit
np.dot(A, x)

In [ ]:
%%timeit
dot_mv(A, x, y)
y

### Try to Get Basic NOGIL Functionality Working
* implement a linear search function to see if a given item is in a user's items
* implement C arrays to hold both each user's number of items and each user's set of items
* all of this must be able to be used in a NOGIL block to parallelize the eventual training loop

In [ ]:
%%cython --annotate

cimport cython
from libc.stdlib cimport malloc, free


cdef int lsearch(int item, int *items, int length) nogil:
    """linear search for a given item in a sorted array of items"""
    
    cdef int i
    for i in range(length):
        if item == items[i]:
            return True
    return False


def process(dict p_user_items):
    """create a C array and try to use it in a [nogil] block"""
    
    # declare all C variables
    cdef int user
    cdef int item
    cdef int n_users
    cdef int *n_items
    cdef int **c_user_items
    
    # count the total users and number of items for each user
    n_users = len(p_user_items)
    n_user_items = {user: len(items) for user, items in p_user_items.items()}
    
    # initialize the [n_items] and [c_user_items] C arrays
    n_items = <int*>malloc(n_users * sizeof(int))
    c_user_items = <int**>malloc(n_users * sizeof(int*))
    
    # fill the C arrays
    for user in range(n_users):
        
        # fill the item count array for each user
        n_items[user] = n_user_items[user]
        
        # allocate the dynamic item set array for each user
        c_user_items[user] = <int*>malloc(n_items[user] * sizeof(int))
        
        # fill the item set array for each user
        for item in range(n_items[user]):
            c_user_items[user][item] = p_user_items[user][item]
            
    # confirm C array values in a NOGIL block
    with nogil:
        for user in range(n_users):
            res = lsearch(9, c_user_items[user], n_items[user])
            for item in range(n_items[user]):
                c_user_items[user][item] += 10
                with gil:
                    print("user:", user, "item:", c_user_items[user][item], "number of items:", n_items[user], "found:", res)
                
    # free the dynamic item arrays for each user 
    for user in range(n_users):
        free(c_user_items[user])
        
    # free the item count and top-level user items arrays
    free(n_items)
    free(c_user_items)
        


In [ ]:
repo_root = "/Users/ericlundquist/Repos/rankfm"
data_path = os.path.join(repo_root, "data/examples")
print("\n".join([repo_root, data_path]))

In [ ]:
interactions = pd.read_csv(os.path.join(data_path, 'ML_1M_RATINGS.csv'))
interactions.info()

In [ ]:
interactions['user_id'] = interactions['user_id'] - 1
interactions['item_id'] = interactions['item_id'] - 1
interactions = interactions.sort_values(['user_id', 'item_id']).groupby('user_id')['item_id'].apply(np.array, dtype=np.int32).to_dict()
print(len(interactions))
interactions[0]

In [ ]:
p_user_items = {
    0: np.array([9, 6, 3], dtype=np.int32),
    1: np.array([2, 4, 6, 8, 10], dtype=np.int32)
}
p_user_items

In [ ]:
%%time
process(p_user_items)

# **START EXAMPLE CODE FROM ONLINE RESOURCES**

### Chapter 1 - Cython Essentials

#### Sample Python Program

In [ ]:
def p_fib(n):
    a, b = 0.0, 1.0
    for i in range(n):
        a, b = a + b, a
    return a

#### Equivalent Cython Program

In [ ]:
%%cython --annotate

def c_fib(int n):
    cdef int i
    cdef double a=0.0, b=1.0
    for i in range(n):
        a, b = a + b, a
    return a

In [ ]:
%%timeit
p_fib(1000)

In [ ]:
%%timeit
c_fib(1000)

### Chapter 9 - Memoryviews

In [ ]:
%%cython

def summer_1(float[:] mv):
    """iterate over a data buffer and compute the sum"""
    
    cdef:
        double d
        ss = 0.0
    for d in mv:
        ss += d
    return ss

#### Faster Version with Direct Access and No Python Overhead
* turn off `boundscheck` and `wraparound`
* use a typed iterator (i) and range(N) for looping
* notice zero python interaction in the function body

In [ ]:
%%cython --annotate

from cython cimport boundscheck, wraparound

@boundscheck(False)
@wraparound(False)
def summer_2(float[:] mv):
    """loop with a typed range and iterator to index directly into the buffer with no Python overhead"""
    
    cdef:
        int i, N
        float ss = 0.0
        
    # easily calculate number of elements
    N = mv.shape[0]
    
    # loop over elements and index directly
    for i in range(N):
        ss += mv[i]
        
    return ss

In [ ]:
to_sum = np.ones(1000000, dtype=np.float32)

In [ ]:
%%timeit
summer_1(to_sum)

In [ ]:
%%timeit
summer_2(to_sum)

In [ ]:
%%cython --annotate

from cython cimport boundscheck, wraparound

@boundscheck(False)
@wraparound(False)
def mv_sum(int[:, ::1] mv):
    """sum the elements of a 2D array"""
    
    cdef int N, M, i, j
    cdef long res = 0
    
    N = mv.shape[0]
    M = mv.shape[1]
    
    for i in range(N):
        for j in range(M):
            res += mv[i,j]
            
    return res

In [ ]:
to_sum = np.ones((100, 100), dtype=np.int32)

In [ ]:
mv_sum(to_sum)

# **START EXAMPLE CODE**

### Bigger Example Using Typed MemoryViews

In [ ]:
%%cython

from array import array
from math import sqrt

from cython cimport cdivision, boundscheck, wraparound
from cpython.array cimport array

@cdivision(True)
cdef inline double A(int i, int j):
    """pulls an element from a predictable infinite matrix"""
    return 1.0 / (((i + j) * (i + j + 1) >> 1) + i + 1)

@boundscheck(False)
@wraparound(False)
cdef void A_times_u(double[::1] u, double[::1] v):
    """matrix-vector dot product - store result in vector [v]"""
    

    cdef int i, j
    cdef u_len = u.shape[0]
    cdef double partial_sum

    for i in range(u_len):
        partial_sum = 0
        for j in range(u_len):
            partial_sum += A(i, j) * u[j]
        v[i] = partial_sum


@boundscheck(False)
@wraparound(False)
cdef void At_times_u(double[::1] u, double[::1] v):
    """matrix-vector dot product on A-transpose"""
    
    cdef:
        int i, j
        u_len = u.shape[0]
        double partial_sum

    for i in range(u_len):
        partial_sum = 0
        for j in range(u_len):
            partial_sum += A(j, i) * u[j]
        v[i] = partial_sum


def B_times_u(u, out, tmp):
    """swapping helper function"""
    
    A_times_u(u, tmp)
    At_times_u(tmp, out)


def spectral_norm(n):
    """compute the final spectral norm"""
    
    u = array("d", [1.0] * n)
    v = array("d", [0.0] * n)
    tmp = array("d", [0.0] * n)

    for _ in range(10):
        B_times_u(u, v, tmp)
        B_times_u(v, u, tmp)

    vBv = vv = 0

    for ue, ve in zip(u, v):
        vBv += ue * ve
        vv  += ve * ve

    return sqrt(vBv / vv)

### Another Example Using Typed MemoryViews

In [ ]:
%%cython --annotate

from cython cimport cdivision, boundscheck, wraparound
import numpy as np
DTYPE = np.intc


cdef int clip(int a, int min_value, int max_value):
    return min(max(a, min_value), max_value)


@boundscheck(False)
@wraparound(False)
def compute_seq(int[:, :] array_1, int[:, :] array_2, int a, int b, int c):

    # declare loop ranges as C-int types
    cdef Py_ssize_t x_max = array_1.shape[0]
    cdef Py_ssize_t y_max = array_1.shape[1]
    assert tuple(array_1.shape) == tuple(array_2.shape)

    # create a numpy array to hold results and a memory view to access it
    result = np.zeros((x_max, y_max), dtype=DTYPE)
    cdef int[:, :] result_view = result

    # declare temporary loop variables and loop iterators as C types
    cdef int tmp
    cdef Py_ssize_t x, y

    # fast loops in C filling the values of the result array
    for x in range(x_max):
        for y in range(y_max):

            tmp = clip(array_1[x, y], 2, 10)
            tmp = tmp * a + array_2[x, y] * b
            result_view[x, y] = tmp + c

    # return the result which will be a numpy array
    return result

### Parallel Version

In [ ]:
os.environ['gcc'] = 'gcc-9'

In [ ]:
%%cython --compile-args=/openmp --link-args=/openmp --force
cimport cython
cimport openmp

import cython.parallel as cp
from cython.parallel import parallel, prange

import numpy as np
cimport cython

ctypedef fused my_type:
    int
    double
    long long


# We declare our plain c function nogil
cdef my_type clip(my_type a, my_type min_value, my_type max_value) nogil:
    return min(max(a, min_value), max_value)


@cython.boundscheck(False)
@cython.wraparound(False)
def compute_par(my_type[:, ::1] array_1, my_type[:, ::1] array_2, my_type a, my_type b, my_type c):

    cdef Py_ssize_t x_max = array_1.shape[0]
    cdef Py_ssize_t y_max = array_1.shape[1]

    assert tuple(array_1.shape) == tuple(array_2.shape)

    if my_type is int:
        dtype = np.intc
    elif my_type is double:
        dtype = np.double
    elif my_type is cython.longlong:
        dtype = np.longlong

    result = np.zeros((x_max, y_max), dtype=dtype)
    cdef my_type[:, ::1] result_view = result

    cdef my_type tmp
    cdef Py_ssize_t x, y

    # We use prange here.
    for x in prange(x_max, nogil=True):
        for y in range(y_max):

            tmp = clip(array_1[x, y], 2, 10)
            tmp = tmp * a + array_2[x, y] * b
            result_view[x, y] = tmp + c

    return result

In [ ]:
import numpy as np
array_1 = np.random.uniform(0, 1000, size=(3000, 2000)).astype(np.intc)
array_2 = np.random.uniform(0, 1000, size=(3000, 2000)).astype(np.intc)
a = 4
b = 3
c = 9

In [ ]:
%timeit compute_seq(array_1, array_2, a, b, c)

In [ ]:
%timeit compute_par(array_1, array_2, a, b, c)

### Parallel Programming with PRANGE()

In [ ]:
%%cython

# distutils: extra_compile_args = -fopenmp
# distutils: extra_link_args = -fopenmp

from cython cimport boundscheck, wraparound
from cython.parallel cimport prange

import numpy as np

cdef inline double norm2(double complex z) nogil:
    return z.real * z.real + z.imag * z.imag


cdef int escape(double complex z, double complex c, double z_max, int n_max) nogil:

    cdef:
        int i = 0
        double z_max2 = z_max * z_max

    while norm2(z) < z_max2 and i < n_max:
        z = z * z + c
        i += 1

    return i


@boundscheck(False)
@wraparound(False)
def calc_julia(int resolution, double complex c, double bound=1.5, double z_max=4.0, int n_max=1000):

    cdef:
        double step = 2.0 * bound / resolution
        int i, j
        double complex z
        double real, imag
        int[:, ::1] counts

    counts = np.zeros((resolution+1, resolution+1), dtype=np.int32)

    for i in prange(resolution + 1, nogil=True, schedule='static', chunksize=1):
        real = -bound + i * step
        for j in range(resolution + 1):
            imag = -bound + j * step
            z = real + imag * 1j
            counts[i,j] = escape(z, c, z_max, n_max)

    return np.asarray(counts)

@boundscheck(False)
@wraparound(False)
def julia_fraction(int[:,::1] counts, int maxval=1000):
    cdef:
        int total = 0
        int i, j, N, M
        
    N = counts.shape[0] 
    M = counts.shape[1]

    for i in prange(N, nogil=True):
        for j in range(M):
            if counts[i,j] == maxval:
                total += 1
    return total / float(counts.size)